In [1]:
# Basic dictionary-based SMT for word-by-word translation

def translate(sentence, dictionary):
    translated_sentence = []
    words = sentence.split()

    for word in words:
        translated_word = dictionary.get(word.lower(), word)  # Use word from dictionary or original word
        translated_sentence.append(translated_word)

    return " ".join(translated_sentence)

# Example dictionary (English to French)
dictionary = {
    "hello": "bonjour",
    "world": "monde",
    "goodbye": "au revoir"
}

sentence = "Hello world"
translated_sentence = translate(sentence, dictionary)
print(f"Original: {sentence}")
print(f"Translated: {translated_sentence}")


Original: Hello world
Translated: bonjour monde


In [2]:
# 2. Attention Mechanism in Neural Machine Translation (NMT) using PyTorch

import torch
import torch.nn as nn
import torch.optim as optim

# Define an encoder-decoder architecture with attention mechanism

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size, hidden_size)

    def forward(self, encoder_outputs, decoder_hidden):
        energy = torch.tanh(self.attn(encoder_outputs))
        attention_scores = torch.sum(energy * decoder_hidden, dim=-1)
        return attention_scores

class NMTModel(nn.Module):
    def __init__(self, input_dim, output_dim, emb_size, hidden_size):
        super(NMTModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_size)
        self.encoder = nn.LSTM(emb_size, hidden_size)
        self.decoder = nn.LSTM(emb_size, hidden_size)
        self.attention = Attention(hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_dim)

    def forward(self, source, target):
        embedded_source = self.embedding(source)
        encoder_outputs, (hidden, cell) = self.encoder(embedded_source)

        embedded_target = self.embedding(target)
        decoder_outputs, _ = self.decoder(embedded_target, (hidden, cell))

        attention_scores = self.attention(encoder_outputs, decoder_outputs)
        context_vector = torch.bmm(attention_scores.unsqueeze(1), encoder_outputs)

        output = self.fc_out(context_vector.squeeze(1))
        return output




In [3]:
# 3. Using Pre-trained GPT Model for English to French Translation
from transformers import MarianMTModel, MarianTokenizer

# Load pre-trained model and tokenizer for English to French translation
model_name = 'Helsinki-NLP/opus-mt-en-fr'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Example translation
english_text = "Hello, how are you?"
french_translation = translate(english_text)
print(f"French Translation: {french_translation}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


French Translation: Bonjour, comment allez-vous ?


In [4]:
# 4. Generate a Short Poem Using GPT-2 for a Specific Theme (e.g., "Nature")
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_poem(theme):
    prompt = f"Write a short poem about {theme}:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
    poem = tokenizer.decode(output[0], skip_special_tokens=True)
    return poem

# Example poem generation
theme = "Nature"
poem = generate_poem(theme)
print(poem)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write a short poem about Nature:

The poem is a poem of the nature of Nature.
. . .
 (The poet is the poet, and the poem the writer.)
, . (the poet and writer. The poem and


In [5]:
# 5. Implement a Basic Reinforcement Learning Setup for Text Generation Using PyTorch's Reward Function
import torch
import torch.nn as nn
import torch.optim as optim

# Define a basic text generation model (RNN for simplicity)
class TextGenerationModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super(TextGenerationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.RNN(emb_size, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output)
        return output

# Define reward-based learning
def reward_function(output_text):
    # Example reward based on some heuristic
    reward = 0
    if "happy" in output_text:
        reward += 1
    return reward

# Example setup for training
def train(model, optimizer, criterion, input_seq, target_seq):
    model.train()
    optimizer.zero_grad()
    output = model(input_seq)
    loss = criterion(output.view(-1, output.size(-1)), target_seq.view(-1))
    loss.backward()
    optimizer.step()
    return loss.item()

# Example of using reward
input_seq = torch.tensor([1, 2, 3, 4])  # Example input
target_seq = torch.tensor([2, 3, 4, 5])  # Example target sequence

model = TextGenerationModel(vocab_size=10, emb_size=5, hidden_size=10)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

loss = train(model, optimizer, criterion, input_seq, target_seq)
generated_text = "This is a happy day"
reward = reward_function(generated_text)

print(f"Loss: {loss}, Reward: {reward}")


Loss: 2.495262622833252, Reward: 1


In [6]:
# 6. Create a Simple Multimodal Generative Model for Image Captioning

import torch
from torch import nn
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load pre-trained image captioning model (BLIP)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_path):
    image = processor(images=image_path, return_tensors="pt").pixel_values
    output = model.generate(image)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# Example usage
image_path = 'image.jpg'  # Provide path to an image
caption = generate_caption(image_path)
print(f"Generated Caption: {caption}")


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

ValueError: Invalid image type. Expected either PIL.Image.Image, numpy.ndarray, torch.Tensor, tf.Tensor or jax.ndarray, but got <class 'str'>.

In [ ]:
# 7. Evaluate Bias in Generated Content (GPT-3 or GPT-4)
from openai import OpenAI
import openai

openai.api_key = 'your-api-key'

def evaluate_bias(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",  # Or another GPT-3 variant
        prompt=prompt,
        max_tokens=50
    )
    return response.choices[0].text.strip()

# Example with a potentially sensitive prompt
prompt = "What is the role of women in technology?"
output = evaluate_bias(prompt)
print(output)


In [8]:
# 8. Create a Simple Neural Machine Translation Model with PyTorch for English to German


import torch
import torch.nn as nn
import torch.optim as optim

# Example for a simple seq2seq NMT Model

class Seq2SeqModel(nn.Module):
    def __init__(self, input_dim, output_dim, emb_size, hidden_size):
        super(Seq2SeqModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_size)
        self.encoder = nn.LSTM(emb_size, hidden_size)
        self.decoder = nn.LSTM(emb_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_dim)

    def forward(self, source, target):
        embedded_source = self.embedding(source)
        encoder_outputs, (hidden, cell) = self.encoder(embedded_source)

        embedded_target = self.embedding(target)
        decoder_outputs, _ = self.decoder(embedded_target, (hidden, cell))

        output = self.fc_out(decoder_outputs)
        return output
